### Hydraulic Design Calculations

The following calculations determine the required vessel dimensions based on hydraulic constraints:

In [ ]:
# Design parameters (from simulation inputs)
Q_m3_hr = design.get('flow_m3_hr', 100)      # Design flow rate in m³/hr
SV_hr = 16                                    # Service velocity (bed volumes/hour)
LV_max_m_hr = 25                             # Maximum linear velocity in m/hr

#### Step 1: Bed Volume Calculation

In [ ]:
# Bed volume calculation using handcalcs
# Note: If %%render doesn't work, calculations will still show values
try:
    get_ipython().run_cell_magic('render', '', '# Bed volume calculation\\nV_bed_m3 = Q_m3_hr / SV_hr\\nV_bed_L = V_bed_m3 * 1000')
except:
    # Fallback if handcalcs not available
    V_bed_m3 = Q_m3_hr / SV_hr
    V_bed_L = V_bed_m3 * 1000
    print(f"V_bed_m3 = Q_m3_hr / SV_hr = {Q_m3_hr} / {SV_hr} = {V_bed_m3:.2f} m³")
    print(f"V_bed_L = V_bed_m3 * 1000 = {V_bed_m3:.2f} * 1000 = {V_bed_L:.0f} L")

#### Step 2: Minimum Cross-Sectional Area

In [ ]:
%%render
# Minimum area for linear velocity
A_min_m2 = Q_m3_hr / LV_max_m_hr

#### Step 3: Vessel Diameter Selection

In [ ]:
from math import pi, sqrt, ceil

%%render
# Minimum diameter from area
D_min_m = sqrt(4 * A_min_m2 / pi)

In [ ]:
# Round up to standard diameter
D_selected_m = min(2.4, ceil(D_min_m * 10) / 10)  # Round to nearest 0.1m, max 2.4m

In [ ]:
%%render
# Actual cross-sectional area
A_actual_m2 = pi * (D_selected_m / 2) ** 2

# Actual linear velocity
LV_actual_m_hr = Q_m3_hr / A_actual_m2

#### Step 4: Bed Depth Calculation

In [ ]:
%%render
# Bed depth from volume and area
h_bed_m = V_bed_m3 / A_actual_m2

In [ ]:
# Apply minimum bed depth constraint and round
h_bed_min_m = 0.75
h_bed_final_m = max(h_bed_min_m, ceil(h_bed_m * 20) / 20)  # Round to 0.05m

In [ ]:
%%render
# Recalculate actual bed volume
V_bed_actual_m3 = A_actual_m2 * h_bed_final_m

# Service velocity in BV/hr
SV_actual_hr = Q_m3_hr / V_bed_actual_m3

#### Step 5: Freeboard and Total Height

In [ ]:
%%render
# Freeboard for backwash expansion
expansion_factor = 1.0
h_freeboard_m = h_bed_final_m * expansion_factor

# Total vessel height
h_distributor_m = 0.1
h_vessel_m = h_bed_final_m + h_freeboard_m + h_distributor_m

# Aspect ratio
aspect_ratio = h_bed_final_m / D_selected_m

#### Vessel Design Summary

In [ ]:
# Summary tables
vessel_summary = [
    ['Vessel Diameter', f"{D_selected_m:.2f} m"],
    ['Bed Depth', f"{h_bed_final_m:.2f} m"],
    ['Bed Volume', f"{V_bed_actual_m3:.2f} m³"],
    ['Freeboard Height', f"{h_freeboard_m:.2f} m"],
    ['Total Vessel Height', f"{h_vessel_m:.2f} m"],
    ['Linear Velocity', f"{LV_actual_m_hr:.1f} m/hr"],
    ['Service Velocity', f"{SV_actual_hr:.1f} BV/hr"],
    ['Aspect Ratio (L/D)', f"{aspect_ratio:.2f}"]
]

df_vessel = pd.DataFrame(vessel_summary, columns=['Parameter', 'Value'])
display(df_vessel.style.set_caption('SAC Vessel Design Summary'))

# Design constraints
constraints = [
    ['Linear Velocity Limit', '25 m/hr'],
    ['Actual Linear Velocity', f"{LV_actual_m_hr:.1f} m/hr"],
    ['Minimum Bed Depth', '0.75 m'],
    ['Actual Bed Depth', f"{h_bed_final_m:.2f} m"],
    ['Container Diameter Limit', '2.4 m'],
    ['Actual Diameter', f"{D_selected_m:.2f} m"]
]

df_constraints = pd.DataFrame(constraints, columns=['Parameter', 'Value'])
display(df_constraints.style.set_caption('Design Constraints'))